In [1]:
import pandas as pd
import numpy as np

In [2]:
df_full = pd.read_pickle('../../Data/block4_concat_lane.pkl')  \
    .set_index('edge_id', append=True) \
    .reorder_levels((0,1,3,2))

df_full.head()

lat        lon    speed  \
file_name id edge_id             time                                   
4_1       1  250699362_250699984 42.00  37.982746  23.732961  11.9046   
                                 42.04  37.982746  23.732963  11.8975   
                                 42.08  37.982747  23.732964  11.8919   
                                 42.12  37.982748  23.732965  11.8871   
                                 42.16  37.982748  23.732966  11.8831   

                                        lon_acc  lat_acc  type  traveled_d  \
file_name id edge_id             time                                        
4_1       1  250699362_250699984 42.00  -0.1145   0.0138  Taxi      182.37   
                                 42.04  -0.1007   0.0147  Taxi      182.37   
                                 42.08  -0.0918   0.0157  Taxi      182.37   
                                 42.12  -0.0869   0.0167  Taxi      182.37   
                                 42.16  -0.0784   0.0176  Taxi      182.37   

                                        avg_speed   bearing  \
file_name id edge_id             time                         
4_1       1  250699362_250699984 42.00   9.740748  1.570795   
                                 42.04   9.740748  0.168572   
                                 42.08   9.740748  0.168573   
                                 42.12   9.740748  1.570796   
                                 42.16   9.740748  0.328080   

                                        nearest_edge_start_node  ...  dir  \
file_name id edge_id             time                            ...        
4_1       1  250699362_250699984 42.00                250699362  ...    0   
                                 42.04                250699362  ...    0   
                                 42.08                250699362  ...    0   
                                 42.12                250699362  ...    0   
                                 42.16                250699362  ...    0   

                                        xtrack_dist  time_stamp  \
file_name id edge_id             time                             
4_1       1  250699362_250699984 42.00    -1.883401       42.00   
                                 42.04    -1.980795       42.04   
                                 42.08    -1.937041       42.08   
                                 42.12    -1.893287       42.12   
                                 42.16    -1.941984       42.16   

                                        edge_progress_intervals     len  \
file_name id edge_id             time                                     
4_1       1  250699362_250699984 42.00                      0.3  97.581   
                                 42.04                      0.3  97.581   
                                 42.08                      0.3  97.581   
                                 42.12                      0.3  97.581   
                                 42.16                      0.3  97.581   

                                        lanes  node_veh_dist  edge_seg  \
file_name id edge_id             time                                    
4_1       1  250699362_250699984 42.00    5.4      29.814330       1.0   
                                 42.04    5.4      29.674830       1.0   
                                 42.08    5.4      29.537753       1.0   
                                 42.12    5.4      29.400718       1.0   
                                 42.16    5.4      29.330986       1.0   

                                        vehicle_density  avg_surr_speed  
file_name id edge_id             time                                    
4_1       1  250699362_250699984 42.00                7       10.464171  
                                 42.04                7       10.457843  
                                 42.08                7       10.452857  
                                 42.12                7       10.448586  
                                 42.16        

In [3]:
df_full['xtrack_diff'] = df_full.xtrack_dist \
    .groupby(df_full.index.names[-1]) \
    .apply(lambda x: (x - x.shift(-1)).fillna(0))
df_full['xtrack_diff']

file_name  id    edge_id               time  
4_1        1     250699362_250699984   42.00    -4.667009
                                       42.04    -4.764403
                                       42.08    -4.720649
                                       42.12    -4.676895
                                       42.16    -4.725592
                                                   ...   
1_3        2116  300400248_8446047162  921.64    0.000000
                                       921.68    0.000000
                                       921.72    0.000000
                                       921.76    0.000000
                                       921.80    0.000000
Name: xtrack_diff, Length: 2997687, dtype: float64

In [4]:
# helper functions

def speed_ratio(grp, min_speed=0):
    return len(grp[grp.speed > min_speed]) / len(grp)

def pivot(A, col_factor):
    c = A.shape[1]
    if A.size < col_factor*c:
        return None
    r_new = A.size // (col_factor*c)
    A = A[:col_factor*r_new]
    return A.to_numpy().reshape(r_new, col_factor*c)

In [5]:
# filtering thresholds
min_traj_len = 300
min_speed_ratio = 0.75

# filter out unwanted trajectories
df = df_full.groupby(df_full.index.names[:-1]) \
            .filter(lambda grp: (len(grp) >= min_traj_len) & (speed_ratio(grp) >= min_speed_ratio))


# features to aggregate (only using mean)
feature_list = ['xtrack_diff','speed','vehicle_density','lat_acc','lon_acc']


# factor by which the number of columns will increase after pivoting
col_factor = 2


# size of aggregating window in seconds
window = '10S'


# function for groupby.apply that will downsample a trajectory and pivot it
def f(grp):
    grp = grp.reset_index(level=(0,1,2), drop=True)
    grp.index = pd.TimedeltaIndex(grp.index,unit='s')
    grp = grp.resample('10S').mean().reset_index(drop=True)
    return pd.DataFrame(pivot(grp,2))


df = df[feature_list] \
    .groupby(df[feature_list].index.names[:-1]) \
    .apply(f) \
    .dropna() \
    .reset_index(level=-1, drop=True)

df.columns = [feature+'_'+str(i) for i in range(col_factor) for feature in feature_list]
df

xtrack_diff_0    speed_0  \
file_name id   edge_id                                          
1_2       4    250709162_250709255        1.421514  11.460398   
          6    250691795_250699359        1.027654  22.235906   
               250691795_250699359        1.886410  14.609982   
          12   250691795_250699359        2.274013  34.174696   
          14   250691795_250699359        4.354276  23.846692   
...                                            ...        ...   
4_3       1678 250699362_250699984       -2.892673  21.037902   
          1685 300400248_8446047162       1.574220  25.057762   
          1686 250699362_250699984        0.872589  21.843258   
          1687 250708556_250708641       -1.068252  17.845259   
          1693 250699362_250699984       -1.504673  20.158099   

                                     vehicle_density_0  lat_acc_0  lon_acc_0  \
file_name id   edge_id                                                         
1_2       4    250709162_250709255               1.504   0.009466   0.304560   
          6    250691795_250699359               2.076   0.524386   0.057630   
               250691795_250699359               4.208   0.034041   0.711512   
          12   250691795_250699359               3.036   0.040916  -0.466609   
          14   250691795_250699359               3.480   0.028044  -0.103692   
...                                                ...        ...        ...   
4_3       1678 250699362_250699984               1.824  -0.017500  -0.145371   
          1685 300400248_8446047162              1.244   0.012869  -0.168590   
          1686 250699362_250699984               2.556  -0.012612  -0.251995   
          1687 250708556_250708641               1.448  -0.011919  -0.713388   
          1693 250699362_250699984               1.424  -0.009345  -0.284707   

                                     xtrack_diff_1    speed_1  \
file_name id   edge_id                                          
1_2       4    250709162_250709255       -1.338097  23.669427   
          6    250691795_250699359        3.864296   4.888101   
               250691795_250699359       -0.453878  25.746587   
          12   250691795_250699359        1.349243  23.590662   
          14   250691795_250699359        5.710396  21.780994   
...                                            ...        ...   
4_3       1678 250699362_250699984       -4.579862   2.838055   
          1685 300400248_8446047162       2.703879  22.356392   
          1686 250699362_250699984        1.093531   3.361691   
          1687 250708556_250708641        0.685400   0.047866   
          1693 250699362_250699984       -0.633495   3.996164   

                                     vehicle_density_1  lat_acc_1  lon_acc_1  
file_name id   edge_id                                                        
1_2       4    250709162_250709255            1.000000   0.148580   0.183004  
          6    250691795_250699359            4.352000   0.016775  -0.517253  
               250691795_250699359            3.838710  -0.182561   0.214932  
          12   250691795_250699359            4.039216   0.042001   0.300042  
          14   250691795_250699359            3.926087   0.008175   0.114639  
...                                                ...        ...        ...  
4_3       1678 250699362_250699984            2.042654   0.012392  -0.600178  
          1685 300400248_8446047162           1.288462   0.116769   0.018077  
          1686 250699362_250699984            2.000000  -0.009505  -0.732040  
          1687 250708556_250708641            2.223404  -0.000167  -0.079040  
          1693 250699362_250699984            1.916667  -0.029621  -0.905588  

[1624 rows x 10 columns]